In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.utils import extmath
import seaborn as sb
import pandas as pd

from utils import omp, basis_pursuit, haupt_alg, emp_risk, risk, iden, iden_adj, sample_sparse_from_unit_ball

In [ ]:
def run_experiment(num_times, opt_proc, n, m, k, B, var_noise, metrics):
    
    metric_mat = np.zeros([num_times, len(metrics)])
    f_star_mat = np.zeros([num_times, n])
    f_hat_mat = np.zeros([num_times, n])
    
    for i in range(num_times):
        #Synthesize k-sparse ground truth
        f_star = sample_sparse_from_unit_ball(n,m,np.linalg.norm)
        f_star*=n*B**2
        
        #Create Rademacher Matrix
        select_phi = np.random.random([n,k])
        Phi = np.zeros([n,k])
        Phi[select_phi>=0.5] = 1./np.sqrt(n)
        Phi[select_phi<0.5] = -1./np.sqrt(n)
        
        #Create Noise and compute random noisy projections
        w = np.random.normal(0,var_noise,k)
        y = Phi.T.dot(f_star) + w
        
        #Run  optimization procedure
        f_hat = opt_proc(Phi,y)
        
        #Compute metrics of interest
        metric_vec = np.zeros(len(metrics))
        for j,met in enumerate(metrics):
            metric_vec[j] = met(f_hat, f_star, Phi, y)
        
        metric_mat[i] = np.copy(metric_vec)
        f_star_mat[i] = np.copy(f_star)
        f_hat_mat[i] = np.copy(f_hat)
        
        if i % (num_times/10)==0:
            print(100*float(i)/float(num_times), "% of iterations complete")
    
    return metric_mat, f_star_mat, f_hat_mat

In [ ]:
#sparsity
ms = np.array([5,10,20,40,60,80,100])#np.arange(0,100,20)+20

#Phi = n x k
n = 1000
k = 100

#variance of noise
var_noise = 0.01

#boundedness of f
B = 1.

#learning rate
eta = 0.1

#lagrange multiplier
gamma = 5.

#epsilon as given in Haupt '06
eps = 1./(50.*(B+np.sqrt(var_noise))**2)

#iters
iters = 500

#Using identity transform for comparison
opt_proc1 = lambda a, b: haupt_alg(a,b,iden,iden_adj,np.ones(n),iters,eps,practical_adj=0.5)[-1][-1]
opt_proc2 = lambda a, b: basis_pursuit(a,b,iden,iden_adj,np.ones(n),iters,eta,gamma)[-1][-1]

#Comparing statisticial risk, validating sparsity is approximately k
metric1 = lambda f,f_s,p,y: risk(f, f_s, var_noise)
metric2 = lambda f,f_s,p,y: np.sum(f>1.)/float(n)

times_per_exp = 1000

full_metrics_haupt = np.zeros([ms.shape[0],times_per_exp,2])
full_metrics_basis = np.zeros([ms.shape[0],times_per_exp,2])
full_metrics_omp = np.zeros([ms.shape[0],times_per_exp,2])

for i,m in enumerate(ms):
    #OMP is dependent on sparsity
    opt_proc3 = lambda a, b: omp(a.T,b,m)
    
    print("m=", m, "Experiment 1/3")
    metrics_ht, f_stars_ht, f_hats_ht = run_experiment(times_per_exp, opt_proc1, n, m, k, B, var_noise, (metric1, metric2))
    print("m=", m, "Experiment 2/3")
    metrics_bp, f_stars_bp, f_hats_bp = run_experiment(times_per_exp, opt_proc2, n, m, k, B, var_noise, (metric1, metric2))
    print("m=", m, "Experiment 3/3")
    metrics_mp, f_stars_mp, f_hats_mp = run_experiment(times_per_exp, opt_proc3, n, m, k, B, var_noise, (metric1, metric2))

    full_metrics_haupt[i] = np.copy(metrics_ht)
    full_metrics_basis[i] = np.copy(metrics_bp)
    full_metrics_omp[i] = np.copy(metrics_mp)

m= 5 Experiment 1/3
0.0 % of iterations complete


In [ ]:
HAUPT_COL = '#94e4ff'
BASIS_COL = '#ff9e54'
OMP_COL = '#aaffa6'


fig = plt.figure(figsize=(18,10))
ax = plt.axes()
ax.set_xticks([3*i for i in range(ms.shape[0])])
ax.set_xticklabels(["k = " + str(m) for m in ms],fontsize='xx-large')

ax.set_ylabel("Risk",fontsize='xx-large')
                   
for i in range(len(ms)):
    parts_h = plt.violinplot(full_metrics_haupt[i,:,0],positions = [3*i-0.6],showmeans=True)
    parts_b = plt.violinplot(full_metrics_basis[i,:,0],positions = [3*i],showmeans=True)
    parts_o = plt.violinplot(full_metrics_omp[i,:,0],positions = [3*i+0.6],showmeans=True)
    
    for pc in parts_h['bodies']:
        pc.set_facecolor(HAUPT_COL)
        pc.set_edgecolor('grey')
        pc.set_alpha(1)
   
    for pc in parts_b['bodies']:
        pc.set_facecolor(BASIS_COL)
        pc.set_edgecolor('grey')
        pc.set_alpha(1)
        
    for pc in parts_o['bodies']:
        pc.set_facecolor(OMP_COL)
        pc.set_edgecolor('grey')
        pc.set_alpha(1)
    
    for partname in ('cbars','cmins','cmaxes','cmeans'):
        vp = parts_h[partname]
        vp.set_edgecolor('grey')
        vp.set_linewidth(1)
        vp = parts_b[partname]
        vp.set_edgecolor('grey')
        vp.set_linewidth(1)
        vp = parts_o[partname]
        vp.set_edgecolor('grey')
        vp.set_linewidth(1)


labels = ['Haupt Recon.', 'Basis Pursuit Recon.', 'OMP Recon.']
patches = [mpatches.Patch(color=HAUPT_COL),  mpatches.Patch(color=BASIS_COL), mpatches.Patch(color=OMP_COL)]

plt.legend(patches,labels,loc='lower right',fontsize='xx-large')

plt.title("Risk Comparison",fontsize='xx-large')

plt.savefig("RISK")

In [ ]:
HAUPT_COL = '#94e4ff'
BASIS_COL = '#ff9e54'
OMP_COL = '#aaffa6'


fig = plt.figure(figsize=(18,10))
ax = plt.axes()
ax.set_xticks([3*i for i in range(ms.shape[0])])
ax.set_xticklabels(["k = " + str(m) for m in ms],fontsize='xx-large')

ax.set_ylabel("Sparsity",fontsize='xx-large')
                   
for i in range(len(ms)):
    parts_h = plt.violinplot(full_metrics_haupt[i,:,1],positions = [3*i-0.6],showmeans=True)
    parts_b = plt.violinplot(full_metrics_basis[i,:,1],positions = [3*i],showmeans=True)
    parts_o = plt.violinplot(full_metrics_omp[i,:,1],positions = [3*i+0.6],showmeans=True)
    
    for pc in parts_h['bodies']:
        pc.set_facecolor(HAUPT_COL)
        pc.set_edgecolor('grey')
        pc.set_alpha(1)
   
    for pc in parts_b['bodies']:
        pc.set_facecolor(BASIS_COL)
        pc.set_edgecolor('grey')
        pc.set_alpha(1)
        
    for pc in parts_o['bodies']:
        pc.set_facecolor(OMP_COL)
        pc.set_edgecolor('grey')
        pc.set_alpha(1)
    
    for partname in ('cbars','cmins','cmaxes','cmeans'):
        vp = parts_h[partname]
        vp.set_edgecolor('grey')
        vp.set_linewidth(1)
        vp = parts_b[partname]
        vp.set_edgecolor('grey')
        vp.set_linewidth(1)
        vp = parts_o[partname]
        vp.set_edgecolor('grey')
        vp.set_linewidth(1)


labels = ['Haupt Recon.', 'Basis Pursuit Recon.', 'OMP Recon.']
patches = [mpatches.Patch(color=HAUPT_COL),  mpatches.Patch(color=BASIS_COL), mpatches.Patch(color=OMP_COL)]

plt.legend(patches,labels,loc='lower right',fontsize='xx-large')

plt.title("Sparsity Ratio Comparison (Verification)",fontsize='xx-large')

plt.savefig("SPARSE")